In [1]:
import soundfile as sf
import random
import numpy as np
import os
import librosa

In [2]:
output_length = 600
sampling_rate = 44100
speakers = ['3607', '7113', '89']

In [3]:
def loadAudio(path, sampling_rate):
    audiodata, sr = sf.read(path)
    if not sr == sampling_rate:
        audiodata = librosa.resample(audiodata, sr, sampling_rate)
    return audiodata

In [4]:
def makeAudio():
    global output_length
    global sampling_rate
    global speakers
    
    total_length = output_length * sampling_rate
    
    true_files = []
    false_files = []
    
    random.shuffle(true_files)
    random.shuffle(false_files)
    
    audio = []
    audio1 = []
    audio2 = []
    audio3 = []
    audio4 = []
    
    labels = []
    labels1 = []
    labels2 = []
    labels3 = []
    labels4 = []
    
    convlabels = []
    
    for filename in os.listdir('./Datasets/chime-home/True'):
        if filename.split('-')[0] in speakers:
            true_files.append(filename)
            
    for filename in os.listdir('./Datasets/chime-home/False'):
        false_files.append(filename)
    
    counter = 0
    
    rando = random.randint(0, 100) 
    
    wanted_length_1 = (total_length / 2) * (rando / 100)
    wanted_length_2 = (total_length / 2) * (1 - (rando / 100))
        
    while len(audio1) < wanted_length_1:
        current_file = true_files[counter % len(true_files)]
        
        audiosamples = loadAudio('./Datasets/chime-home/True/' + current_file, sampling_rate)
        max_length = len(audiosamples) - (len(audiosamples) % sampling_rate)
        
        audiodata = audiosamples[:max_length]

        audio1.extend(audiodata)
        labels1.extend(np.repeat([current_file.split('-')[0]], int(len(audiodata) / sampling_rate)).tolist())
        
        rand = random.randint(0, 100) 
        if rand < 10:
            noisesamples = loadAudio('./Datasets/chime-home/False/' + false_files[(counter + rand) % len(false_files)], sampling_rate)
            max_length = len(audiosamples) - (len(audiosamples) % sampling_rate)
            
            noise = noisesamples[:max_length]
            
            audio1.extend(noise)
            labels1.extend(np.repeat([False], int(len(noise) / sampling_rate)).tolist())
            
        counter += 1
        print('progress: ' + str((((len(audio1) + len(audio2) + len(audio3) + len(audio4)) / sampling_rate) / output_length) * 100) + '%', end='\r')
        
    while len(audio2) < wanted_length_2:
        current_file = false_files[counter % len(false_files)]
        
        audiosamples = loadAudio('./Datasets/chime-home/False/' + current_file, sampling_rate)
        max_length = len(audiosamples) - (len(audiosamples) % sampling_rate)
        
        audiodata = audiosamples[:max_length]
        
        audio2.extend(audiodata)
        labels2.extend(np.repeat([False], int(len(audiodata) / sampling_rate)).tolist())
        
        counter += 1
        print('progress: ' + str((((len(audio1) + len(audio2) + len(audio3) + len(audio4)) / sampling_rate) / output_length) * 100) + '%', end='\r')
        
    while len(audio3) < wanted_length_2:
        current_file = true_files[counter % len(true_files)]
        
        audiosamples = loadAudio('./Datasets/chime-home/True/' + current_file, sampling_rate)
        max_length = len(audiosamples) - (len(audiosamples) % sampling_rate)
        
        audiodata = audiosamples[:max_length]
        
        audio3.extend(audiodata)
        labels3.extend(np.repeat([current_file.split('-')[0]], int(len(audiodata) / sampling_rate)).tolist())
        
        rand = random.randint(0, 100) 
        if rand < 10:
            noisesamples = loadAudio('./Datasets/chime-home/False-/' + false_files[(counter + rand) % len(false_files)], sampling_rate)
            max_length = len(audiosamples) - (len(audiosamples) % sampling_rate)
            
            noise = noisesamples[:max_length]
            
            audio3.extend(noise)
            labels3.extend(np.repeat([False], int(len(noise) / sampling_rate)).tolist())
            
        counter += 1
        print('progress: ' + str((((len(audio1) + len(audio2) + len(audio3) + len(audio4)) / sampling_rate) / output_length) * 100) + '%', end='\r')
        
    while len(audio4) < wanted_length_1:
        current_file = false_files[counter % len(true_files)]
        
        audiosamples = loadAudio('./Datasets/chime-home/False/' + current_file, sampling_rate)
        max_length = len(audiosamples) - (len(audiosamples) % sampling_rate)
        
        audiodata = audiosamples[:max_length]
        
        audio4.extend(audiodata)
        labels4.extend(np.repeat([False], int(len(audiodata) / sampling_rate)).tolist())
        
        counter += 1
        print('progress: ' + str((((len(audio1) + len(audio2) + len(audio3) + len(audio4)) / sampling_rate) / output_length) * 100) + '%', end='\r')
    
    print('\n')
    
    if rand % 4 == 0:
        audio = audio1 + audio2 + audio3 + audio4
        labels = labels1 + labels2 + labels3 + labels4
        convlabels = (
            np.repeat(True, len(labels1)).tolist() +
            np.repeat(False, len(labels2)).tolist() +
            np.repeat(True, len(labels3)).tolist() +
            np.repeat(False, len(labels4)).tolist() )

    elif rand % 4 == 1:
        audio = audio2 + audio1 + audio3 + audio4
        labels = labels2 + labels1 + labels3 + labels4
        convlabels = (            
            np.repeat(False, len(labels2)).tolist() +
            np.repeat(True, len(labels1)).tolist() +
            np.repeat(True, len(labels3)).tolist() +
            np.repeat(False, len(labels4)).tolist() )

    elif rand % 4 == 2:
        audio = audio1 + audio2 + audio4 + audio3
        labels = labels1 + labels2 + labels4 + labels3
        convlabels = (            
            np.repeat(True, len(labels1)).tolist() +
            np.repeat(False, len(labels2)).tolist() +
            np.repeat(False, len(labels4)).tolist() +
            np.repeat(True, len(labels3)).tolist() )

    else:
        audio = audio2 + audio1 + audio4 + audio3
        labels = labels2 + labels1 + labels4 + labels3
        convlabels = (            
            np.repeat(False, len(labels2)).tolist() +
            np.repeat(True, len(labels1)).tolist() +
            np.repeat(False, len(labels4)).tolist() +
            np.repeat(True, len(labels3)).tolist() )
    
    return audio, labels, convlabels

In [5]:
def main():
    global output_length
    global sampling_rate
    audio, labels, convlabels = makeAudio()
    
    print(len(audio), len(labels), len(convlabels))

    sf.write('./conversation_audio.wav', audio, sampling_rate)
    np.save('./conversation_labels.npy', labels)
    np.save('./conversation_convlabels.npy', convlabels)
    
    return audio

In [6]:
audio = main()

progress: 103.16666666666667%

27297900 619 619
['89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '3607', '3607', '3607', '3607', '3607', '3607', '3607', '3607', '3607', '3607', '3607', '3607', '3607', '3607', '3607', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', '89', False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, 